In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist

#Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [4]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
from tensorflow.keras import regularizers
model = Sequential()
model.add(layers.Dropout(0.2,input_shape=(784,)))
model.add(layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,  activation='softmax'))
#display the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1000)              785000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
______________________________________________

In [8]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

In [9]:
hist = model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=100,
                        verbose=1,
                        validation_data=(x_test,y_test))

Epoch 1/100
469/469 [==============================] - 4s 9ms/step - loss: 2.0577 - accuracy: 0.8565 - val_loss: 0.7051 - val_accuracy: 0.9175
Epoch 2/100
469/469 [==============================] - 4s 9ms/step - loss: 0.7612 - accuracy: 0.8930 - val_loss: 0.6555 - val_accuracy: 0.9185
Epoch 3/100
469/469 [==============================] - 4s 9ms/step - loss: 0.7241 - accuracy: 0.8984 - val_loss: 0.5948 - val_accuracy: 0.9409
Epoch 4/100
469/469 [==============================] - 4s 9ms/step - loss: 0.7125 - accuracy: 0.9011 - val_loss: 0.5927 - val_accuracy: 0.9377
Epoch 5/100
469/469 [==============================] - 4s 9ms/step - loss: 0.6867 - accuracy: 0.9046 - val_loss: 0.5666 - val_accuracy: 0.9475
Epoch 6/100
469/469 [==============================] - 4s 9ms/step - loss: 0.6799 - accuracy: 0.9045 - val_loss: 0.5519 - val_accuracy: 0.9474
Epoch 7/100
469/469 [==============================] - 4s 9ms/step - loss: 0.6582 - accuracy: 0.9090 - val_loss: 0.5679 - val_accuracy: 0.9441

Epoch 58/100
469/469 [==============================] - 4s 9ms/step - loss: 0.5553 - accuracy: 0.9114 - val_loss: 0.4236 - val_accuracy: 0.9526
Epoch 59/100
469/469 [==============================] - 4s 9ms/step - loss: 0.5540 - accuracy: 0.9118 - val_loss: 0.4328 - val_accuracy: 0.9479
Epoch 60/100
469/469 [==============================] - 4s 9ms/step - loss: 0.5577 - accuracy: 0.9115 - val_loss: 0.4194 - val_accuracy: 0.9528
Epoch 61/100
469/469 [==============================] - 4s 9ms/step - loss: 0.5546 - accuracy: 0.9113 - val_loss: 0.4168 - val_accuracy: 0.9602
Epoch 62/100
469/469 [==============================] - 4s 9ms/step - loss: 0.5579 - accuracy: 0.9109 - val_loss: 0.4170 - val_accuracy: 0.9572
Epoch 63/100
469/469 [==============================] - 4s 9ms/step - loss: 0.5527 - accuracy: 0.9116 - val_loss: 0.4116 - val_accuracy: 0.9568
Epoch 64/100
469/469 [==============================] - 4s 9ms/step - loss: 0.5570 - accuracy: 0.9111 - val_loss: 0.4206 - val_accuracy:

In [10]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))

313/313 [==============================] - 1s 2ms/step - loss: 0.3954 - accuracy: 0.9578
Test loss 0.3954, accuracy 95.78%


In [11]:
#Save the entire model in model.h5 file
model.save("models/medium_model.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
model = tf.keras.models.load_model('models/medium_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
#saving converted model in "converted_model.tflite" file
open("models/medium_converted_model.tflite", "wb").write(tflite_model)

7185724

In [13]:
model = tf.keras.models.load_model('models/medium_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open("models/medium_converted_quant_model.tflite", "wb").write(tflite_quant_model)

1803984

In [15]:
model = tf.keras.models.load_model('models/medium_model.h5')
model_lite = tf.keras.models.load_model('models/medium_converted_model.tflite')

OSError: SavedModel file does not exist at: models/medium_converted_model.tflite/{saved_model.pbtxt|saved_model.pb}